# SpaceX Data Wrangling & Feature Preparation

## Objective
Clean, standardize, and merge SpaceX launch data collected from
the SpaceX API and Wikipedia into a unified dataset suitable for
exploratory data analysis and machine learning.

This step transforms raw, heterogeneous data sources into a
consistent and analyzable format.


## Input Datasets

- **SpaceX API (raw)**: Launch metadata, rocket IDs, launch sites, outcomes
- **Wikipedia (raw)**: Payload mass, orbit type, mission details

Both datasets were persisted independently to ensure reproducibility
and traceability across the pipeline.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
data_dir = Path("../data/raw")

api_df = pd.read_csv(data_dir / "spacex_launches_raw.csv")
wiki_df = pd.read_csv(data_dir / "spacex_wikipedia_launches_raw.csv")

api_df.shape, wiki_df.shape


((205, 43), (763, 10))

Before merging, both datasets require cleaning and normalization.
Key challenges include:
- Nested structures in API data
- Non-numeric payload values
- Inconsistent naming and missing values


In [3]:
api_cols = [
    "flight_number",
    "name",
    "date_utc",
    "success",
    "rocket",
    "launchpad",
    "payloads",
    "cores"
]

api_df = api_df[api_cols]
api_df.head()


,flight_number,name,date_utc,success,rocket,launchpad,payloads,cores
0,1,FalconSat,2006-03-24T22:30:00.000Z,False,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,['5eb0e4b5b6c3bb0006eeb1e1'],"[{'core': '5e9e289df35918033d3b2623', 'flight'..."
1,2,DemoSat,2007-03-21T01:10:00.000Z,False,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,['5eb0e4b6b6c3bb0006eeb1e2'],"[{'core': '5e9e289ef35918416a3b2624', 'flight'..."
2,3,Trailblazer,2008-08-03T03:34:00.000Z,False,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,"['5eb0e4b6b6c3bb0006eeb1e3', '5eb0e4b6b6c3bb00...","[{'core': '5e9e289ef3591814873b2625', 'flight'..."
3,4,RatSat,2008-09-28T23:15:00.000Z,True,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,['5eb0e4b7b6c3bb0006eeb1e5'],"[{'core': '5e9e289ef3591855dc3b2626', 'flight'..."
4,5,RazakSat,2009-07-13T03:35:00.000Z,True,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,['5eb0e4b7b6c3bb0006eeb1e6'],"[{'core': '5e9e289ef359184f103b2627', 'flight'..."


## Cleaning Payload Mass

Payload mass values extracted from Wikipedia often contain
units, annotations, or missing values.
These values must be converted into numeric kilograms.


In [4]:
payload_candidates = [
    col for col in wiki_df.columns
    if "payload" in col.lower()
]

payload_candidates

['payload[k]', 'payload_mass']

In [5]:
payload_col = payload_candidates[0]
payload_col

'payload[k]'

In [6]:
wiki_df[payload_col] = (
    wiki_df[payload_col]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.extract(r"(\d+\.?\d*)")[0]
    .astype(float)
)

wiki_df[payload_col].describe()

count     725.000000
mean       95.707172
std       367.055309
min         0.000000
25%         6.000000
50%        15.000000
75%        26.000000
max      4000.000000
Name: payload[k], dtype: float64

## Column Normalization

Column names are standardized to ensure consistency
across datasets prior to merging.


In [7]:
wiki_df = wiki_df.rename(columns={
    payload_col: "payload_mass_kg",
    "orbit": "orbit"
})

wiki_df.columns


Index(['flight_no.', 'date_and_time_(utc)', 'version,_booster[j]',
       'launch_site', 'payload_mass_kg', 'payload_mass', 'orbit', 'customer',
       'launch_outcome', 'booster_landing'],
      dtype='object')

## Creating Landing Outcome Label

For downstream analysis, a binary target variable is created:
- `1` → Successful landing
- `0` → Failed landing

This label will be used in both EDA and machine learning models.


In [8]:
api_df["landing_success"] = api_df["success"].fillna(False).astype(int)

api_df["landing_success"].value_counts()


/var/folders/f4/1bx7ntl12p3_b33_lgfy2_t40000gn/T/ipykernel_49750/3753731711.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  api_df["landing_success"] = api_df["success"].fillna(False).astype(int)


landing_success
1    181
0     24
Name: count, dtype: int64

## Data Integration Strategy

Although both the SpaceX API and Wikipedia contain launch-related
information, they do not share a reliable common identifier that
allows consistent record-level merging.

Differences in flight numbering conventions, mission naming,
and editorial formatting prevent a robust one-to-one join.

For this project:
- The SpaceX API is used as the primary analytical dataset for
  exploratory analysis, dashboards, and machine learning.
- Wikipedia data is treated as a complementary source to analyze
  payload mass distributions, orbit types, and qualitative outcomes.

This approach avoids forcing unreliable joins and reflects
best practices when working with heterogeneous real-world data sources.

In [9]:
# Select and clean API dataset (main analytical dataset)
api_clean = api_df.copy()

api_clean["landing_success"] = (
    api_clean["success"]
    .fillna(False)
    .astype(int)
)

api_clean["launch_date"] = pd.to_datetime(api_clean["date_utc"]).dt.date

api_clean = api_clean.dropna(subset=["launch_date"])

api_clean.shape


/var/folders/f4/1bx7ntl12p3_b33_lgfy2_t40000gn/T/ipykernel_49750/4231516662.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(False)


(205, 10)

In [10]:
wiki_clean = wiki_df.copy()

# Clean Wikipedia date field (remove reference annotations like [23])
wiki_clean["launch_date"] = (
    wiki_clean["date_and_time_(utc)"]
    .astype(str)
    .str.replace(r"\[.*?\]", "", regex=True)
    .str.strip()
)

wiki_clean["launch_date"] = pd.to_datetime(
    wiki_clean["launch_date"],
    errors="coerce"
).dt.date

wiki_clean.shape

(763, 11)

## Final Datasets

Due to the lack of a reliable one-to-one identifier between the SpaceX API
and Wikipedia sources, the final data pipeline produces **two clean and
independent datasets**, each optimized for a specific analytical purpose.

### 1. SpaceX API Dataset (Primary)

This dataset is derived from the official SpaceX API and serves as the
**primary analytical dataset** for the project.

It contains:
- Launch metadata (date, rocket, launch site)
- Core and payload configuration
- Binary landing success label (`landing_success`)
- Structured fields suitable for modeling and visualization

This dataset is used for:
- Exploratory data analysis
- Interactive dashboards (Plotly Dash)
- Machine learning classification models

---

### 2. Wikipedia Dataset (Complementary)

This dataset is derived from Wikipedia launch tables and provides
**complementary mission-level insights**.

It contains:
- Payload mass (kg)
- Orbit type
- Customer information
- Qualitative launch and landing outcomes

This dataset is used for:
- Payload and orbit distribution analysis
- Cross-validation of trends observed in API data
- Descriptive and contextual visualizations

---

### Design Decision

Separating the datasets avoids forcing unreliable joins and reflects
best practices when working with heterogeneous real-world data sources.
This approach ensures data integrity while preserving analytical value
from both sources.


In [11]:
output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

api_clean.to_csv(output_dir / "spacex_api_clean.csv", index=False)
wiki_clean.to_csv(output_dir / "spacex_wikipedia_clean.csv", index=False)

## Next Steps

The cleaned dataset will be used to perform:
- Exploratory data analysis using visualization and SQL
- Interactive analytics using Plotly and Dash
- Machine learning classification models
